In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
import dill

# define figure
fig = None

## Versuchsbeschreibung
Die technische Grauwertkonvertierung wertet alle Farbkomponenten RGB gleich.
Die wahrnehmungsgerechte Grauwertkonvertierung gewichtet die Farbkomponenten RGB unterschiedlich (siehe Formel für Y im Skript).
Vergleichen Sie die beiden Grauwertkonvertierungen zunächst an dem Testbild "Rot-Grün-Blau Streifen" und dann an den anderen Bildern oder an den eigenen Bildern. 

In [2]:
if fig:
    plt.close(fig)

# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()

def rgb_to_gray_colorimetric(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return 0.3 * image_in[:, :, 0] + 0.59 * image_in[:, :, 1] + 0.11 * image_in[:, :, 2]

def rgb_to_gray_average(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return np.mean(image_in, axis=2)

def display_images(image_name):  
    # read image
    image = cv2.imread(dict_images[image_name])
    
    # image in RGB (just switching dims)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # transform images
    image_gray_average = rgb_to_gray_average(image_rgb)
    image_gray_colorimetric = rgb_to_gray_colorimetric(image_rgb)
    
    # plot images
    image_1.value = convert_image(image_rgb)
    image_2.value = convert_image(image_gray_average)
    image_3.value = convert_image(image_gray_colorimetric)

    
def change_val(change):
    if change['type'] == 'change' and change['name'] == 'value':
        display_images(change['new'])


select = widgets.Select(
    options=list(dict_images.keys()),
    value='birne',
    description='Dateiliste:')


file = open("images/birne.png", "rb")
image_test = file.read()

image_1 = widgets.Image(value=image_test, format='png', width=300, height=400)
image_2 = widgets.Image(value=image_test, format='png', width=300, height=400)
image_3 = widgets.Image(value=image_test, format='png', width=300, height=400)
label_1 = widgets.Label(value='Ursprüngliches Bild')
label_2 = widgets.Label(value='Technische Grauwertkonvertierung')
label_3 = widgets.Label(value='Wahrnehmungsgerechte Grauwertkonvertierung')

box_1 = widgets.VBox([label_1, image_1])
box_2 = widgets.VBox([label_2, image_2])
box_3 = widgets.VBox([label_3, image_3])

items = [box_1, box_2, box_3]
box = widgets.HBox(items)

display_images('birne')
display(select)
display(box)

select.observe(change_val)

Select(description='Dateiliste:', options=('birne', 'buch', 'fahrzeug', 'klammermaschine', 'mandrill', 'orange…

## 1. Welche Aussagen zur Grauwertkonvertierung treffen zu?

In [3]:
save_checkbox = 'save/checkbox_array_gray_value.dill'

checkbox_array = np.array([False, False, False, False])
# try to load checkbox array
try: 
    with open(save_checkbox, 'rb') as fp:
        checkbox_array = dill.load(fp)
except:
    pass

checkbox_list = ['Sie reduziert den Speicheraufwand eines RGB-Bildes um ca. 45%.',
                 'Die wahrnehmungsgerechte Grauwertkonvertierung berücksichtigt unter anderem, dass der Mensch die Farbe Rot heller wahrnimmt als Grün.', 
                 'Die technische Grauwerkonvertierung führt zu einem besseren Bildkontrast.', 
                 'Die wahrnehmungsgerechte Grauwertkonvertierung berücksichtigt unter anderem, dass der Mensch die Farbe Grün heller wahrnimmt als Blau.']

def callback_checkbox(change):
    if change['type'] == 'change' and change['name'] == 'value':
        checkbox_array[checkbox_list.index(change['owner'].description)] = change['owner'].value
        # save when changed
        with open(save_checkbox, 'wb') as fp:
            dill.dump(checkbox_array, fp)
        
checkbox_dict = {i:widgets.Checkbox(
    value=bool(checkbox_array[i]),
    description=element,
    disabled=False,
    layout = {'width': '950px', 'align_items': 'stretch'}) 
                 for i, element in enumerate(checkbox_list)}

for ele in range(len(checkbox_list)):
    checkbox_dict[ele].observe(callback_checkbox)

display(checkbox_dict[0], checkbox_dict[1], checkbox_dict[2], checkbox_dict[3])

Checkbox(value=False, description='Sie reduziert den Speicheraufwand eines RGB-Bildes um ca. 45%.', layout=Lay…

Checkbox(value=False, description='Die wahrnehmungsgerechte Grauwertkonvertierung berücksichtigt unter anderem…

Checkbox(value=False, description='Die technische Grauwerkonvertierung führt zu einem besseren Bildkontrast.',…

Checkbox(value=False, description='Die wahrnehmungsgerechte Grauwertkonvertierung berücksichtigt unter anderem…

In [4]:
# Autograding answer, please ignore
